In [1]:
import json
import time
import pymongo
from pymongo import MongoClient

import random
import cassiopeia
from cassiopeia import riotapi
import cassiopeia.type.core.common as common

from collections import deque

riotapi.set_region("OCE")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.print_calls(True)
summoner = riotapi.get_summoner_by_name("smilming")
print "Name:" + summoner.name
print "Level:" + str(summoner.level)
print "ID:" + str(summoner.id)


https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/by-name/smilming?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
Name:smilming
Level:30
ID:2780130


In [2]:
masters = riotapi.get_master()

https://oce.api.pvp.net/api/lol/oce/v2.5/league/master?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24&type=RANKED_SOLO_5x5
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/555008,1132033,229379,1153622,627719,1582600,4461579,5540365,497679,7600144,638484,498717,367137,498210,1380903,324274,314888,5540914,1343028,3762231,1530424,578105,4560956,291902,1000512,380994,402614,2922593,319522,620617,724554,297036,2560269,6381137,554066,236630,3210839,4890200,579674,229981?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/285278,397919,5530208,402017,1505382,220270,399473,890994,399995,3700139,222336,453249,343682,774275,1363078,272065,508040,281749,323736,289945,2910362,6070946,620195,276134,329383,1321671,1160364,273070,402607,232626,6851253,303286,411324,293793,915141,431814,447687,632520,582861,272845?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/365776,319185,4802261,302301,369886,276609,3302634,2

In [3]:
def GetTierDivisionValue(tier, division):
    tierMap = {common.Tier.unranked: -1, common.Tier.bronze:0, common.Tier.silver:10, common.Tier.gold:20, common.Tier.platinum:30, common.Tier.diamond:40, common.Tier.master:50, common.Tier.challenger:60}
    divisionMap = {common.Division.one:8, common.Division.two:6, common.Division.three:4, common.Division.four:2, common.Division.five:0}
    tierValue = tierMap[tier]
    divisionValue = divisionMap[division]
    return tierValue + divisionValue
print GetTierDivisionValue(common.Tier.challenger, common.Division.one)

68


In [4]:
print masters[0].division
print type(masters[0].division)
print masters[0].league_points
print type(masters.tier)

Division.one
<enum 'Division'>
231
<enum 'Tier'>


In [5]:
client = MongoClient()
loladb = client.loladb
summoners_collection = loladb.summoners
matches_collection = loladb.matches
leagues_collection = loladb.leagues

In [6]:
#print masters.data.to_json()
#print type(masters)
#league_dict = json.loads(masters.data.to_json())
#print current_summoner.name
#print current_summoner.id
print summoners_collection.count()
print leagues_collection.count()
print matches_collection.count()
#for smn in summoners_collection.find():
    #print smn
#print match_list[0].to_json()
#RecordSummonerLatestMatchId(current_summoner, match_list[0].id)
#print leagues_collection.find_one({})

28
1
16314


In [7]:
def TrimMatchData(match):
    match['timeline'] = None
    
    if match['participants'] != None:
        for i, par in enumerate(match['participantIdentities']):
            par['championId'] = match['participants'][i]['championId']
            par['summonerId'] = par['player']['summonerId']
            par['role'] = match['participants'][i]['timeline']['role']
            par['lane'] = match['participants'][i]['timeline']['lane']
            par['player'] = None
        match['participants'] = None
    
    if match['teams'] != None:
        match['winner'] = -1
        if match['teams'][0]['winner'] == True:
            match['winner'] = 0
        if match['teams'][1]['winner'] == True:
            match['winner'] = 1
        match['teams'] = None

In [13]:
def RecordSummonerLatestMatchId(summoner, matchId):
    summoners_collection.update_one({'id':summoner.id}, {'$set': {'latestMatchId': matchId}})

def RecordSummonerForFullGrab(summoner):
    if summoners_collection.find_one({'id':summoner.id}) == None:
        summoner_dict = dict()
        summoner_dict['name'] = summoner.name
        summoner_dict['id'] = summoner.id
        summoner_dict['summonerLevel'] = summoner.data.summonerLevel
        summoner_dict['revisionDate'] = summoner.data.revisionDate
        summoner_dict['fullGrab'] = True
        summoners_collection.insert_one(summoner_dict)
    else:
        summoners_collection.update_one({'id':summoner.id}, {'$set': {'fullGrab': True}})
        

def IsLatestMatch(match_dict):
    if match_dict['season'] ==  "SEASON2017" or match_dict['season'] ==  "PRESEASON2017" or match_dict['season'] ==  "SEASON2016":
        return True
    else:
        return False

def IsSoloRank(match_type):
    if match_type == 'TEAM_BUILDER_RANKED_SOLO' or match_type == 'TEAM_BUILDER_DRAFT_RANKED_5x5':
        return True
    else:
        return False
    
def IsRequiredMatch(match_dict):
    if IsSoloRank(match_dict['queueType']) and match_dict['matchMode'] == 'CLASSIC' and match_dict['matchType'] == 'MATCHED_GAME':
        return True
    else:
        return False

def RecordMatchesForSummoner(summoner):
    fullGrabbed = False
    latestGrabbedMatchId = 0
    summoner_record = summoners_collection.find_one({'id':summoner.id})
    if summoner_record != None and summoner_record['fullGrab'] == True:
        fullGrabbed = True
        latestGrabbedMatchId = summoner_record['latestMatchId']
        
    match_list = summoner.match_list()
    list_size = len(match_list)
    print 'Match list number:' + str(list_size)
    for match_idx, match in enumerate(match_list):
        print 'processing %d - %d' % (match_idx, list_size)
        if fullGrabbed == True and match.id <= latestGrabbedMatchId:
            print 'Because we already full grabbed this summoner, break the loop while current match is already grabbed'
            if match_idx != 0:
                RecordSummonerLatestMatchId(summoner, match_list[0].id)
                time.sleep( 4 )
                print "wait 4 seconds"
            break
            
        if IsSoloRank(match.data.queue) == False:
            print 'Continue for not solo rank with id ' + str(match.id)
            print "wait 4 seconds"
            time.sleep( 4 )
            if match_idx == list_size - 1:
                #last one
                print 'Recording summoner for a full grab'
                RecordSummonerForFullGrab(summoner)
                RecordSummonerLatestMatchId(summoner, match_list[0].id)
            continue
            
        if matches_collection.find_one({'matchId':match.id}) == None:
            match_data = match.match()
            print "wait 4 seconds"
            time.sleep( 4 )
            match_dict = json.loads(match_data.to_json())
            TrimMatchData(match_dict)
            print match_dict['matchId']
            print match_dict['matchCreation']
            if IsLatestMatch(match_dict):
                if IsRequiredMatch(match_dict):
                    matches_collection.insert_one(match_dict)
                    print 'Match saved to db with id ' + str(match.id)
                else:
                    print 'Match not required with id ' + str(match.id)
            else:
                print 'Break for reaching outdated matches'
                print 'Recording summoner for a full grab'
                RecordSummonerForFullGrab(summoner)
                RecordSummonerLatestMatchId(summoner, match_list[0].id)
                break
        else:
            print 'Found match in db with id ' + str(match.id)

for entry in masters:
    current_summoner = entry.summoner
    try:
        RecordMatchesForSummoner(current_summoner)
    except (AttributeError, cassiopeia.type.api.exception.APIError) as apiError:
        print apiError
        print "Oops!  Error happened. Keep going..."
        print "wait 4 seconds"
        time.sleep( 4 )
    

https://oce.api.pvp.net/api/lol/oce/v2.2/matchlist/by-summoner/372999?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v2.2/matchlist/by-summoner/372999?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
Match list number:772
processing 0 - 772
https://oce.api.pvp.net/api/lol/oce/v2.2/match/2360070795?includeTimeline=True&api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
Server returned error 404 on call: https://oce.api.pvp.net/api/lol/oce/v2.2/match/2360070795?includeTimeline=True&api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
Oops!  Error happened. Keep going...
wait 4 seconds
https://oce.api.pvp.net/api/lol/oce/v2.2/matchlist/by-summoner/222994?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v2.2/matchlist/by-summoner/222994?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v2.2/matchlist/by-summoner/222994?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
Match list number:1478
processing 0 - 1478
Becaus

KeyboardInterrupt: 

In [55]:
match_list = current_summoner.match_list()
print len(match_list)
match = match_list[0].match()
match_dict = json.loads(match.to_json())
TrimMatchData(match_dict)
print json.dumps(match_dict)

https://oce.api.pvp.net/api/lol/oce/v2.2/matchlist/by-summoner/503773?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
812
{"winner": 1, "participantIdentities": [{"player": null, "championId": 64, "participantId": 1, "summonerId": 220201}, {"player": null, "championId": 81, "participantId": 2, "summonerId": 568000}, {"player": null, "championId": 150, "participantId": 3, "summonerId": 479135}, {"player": null, "championId": 7, "participantId": 4, "summonerId": 230726}, {"player": null, "championId": 223, "participantId": 5, "summonerId": 503773}, {"player": null, "championId": 60, "participantId": 6, "summonerId": 411324}, {"player": null, "championId": 201, "participantId": 7, "summonerId": 483862}, {"player": null, "championId": 127, "participantId": 8, "summonerId": 3490137}, {"player": null, "championId": 18, "participantId": 9, "summonerId": 233721}, {"player": null, "championId": 57, "participantId": 10, "summonerId": 365399}], "matchVersion": "6.23.166.2676", "platformId": "OC1", "

In [68]:
print matches_collection.find_one({'matchId':154857985})

{u'matchId': 154857985, u'participantIdentities': [{u'lane': u'BOTTOM', u'championId': 67, u'participantId': 1, u'player': None, u'role': u'DUO_CARRY', u'summonerId': 3762231}, {u'lane': u'MIDDLE', u'championId': 81, u'participantId': 2, u'player': None, u'role': u'SOLO', u'summonerId': 314889}, {u'lane': u'JUNGLE', u'championId': 121, u'participantId': 3, u'player': None, u'role': u'NONE', u'summonerId': 1582600}, {u'lane': u'BOTTOM', u'championId': 40, u'participantId': 4, u'player': None, u'role': u'DUO_SUPPORT', u'summonerId': 444705}, {u'lane': u'TOP', u'championId': 157, u'participantId': 5, u'player': None, u'role': u'SOLO', u'summonerId': 334421}, {u'lane': u'JUNGLE', u'championId': 64, u'participantId': 6, u'player': None, u'role': u'NONE', u'summonerId': 601923}, {u'lane': u'BOTTOM', u'championId': 412, u'participantId': 7, u'player': None, u'role': u'DUO_SUPPORT', u'summonerId': 503773}, {u'lane': u'MIDDLE', u'championId': 103, u'participantId': 8, u'player': None, u'role': 

In [53]:
print current_summoner.id
print current_summoner.name

503773
NADA1001


In [16]:
if leagues_collection.find_one({'name':league_dict['name']}) == None:
    leagues_collection.insert_one(dict(league_dict))
else:
    print 'League already exists'
#print leagues_collection.find_one({name })

League already exists


In [56]:
match_list = summoner.match_list()
print len(match_list)
match = match_list[0].match()
match_without_timeline = json.loads(match.to_json())
        
TrimMatchData(match_without_timeline)
print json.dumps(match_without_timeline)

https://oce.api.pvp.net/api/lol/oce/v2.2/matchlist/by-summoner/2780130?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
1638
{"winner": 1, "participantIdentities": [{"lane": "TOP", "championId": 106, "participantId": 1, "player": null, "role": "SOLO", "summonerId": 230978}, {"lane": "MIDDLE", "championId": 99, "participantId": 2, "player": null, "role": "SOLO", "summonerId": 2780130}, {"lane": "BOTTOM", "championId": 15, "participantId": 3, "player": null, "role": "DUO_CARRY", "summonerId": 5821224}, {"lane": "BOTTOM", "championId": 412, "participantId": 4, "player": null, "role": "DUO_SUPPORT", "summonerId": 3163357}, {"lane": "JUNGLE", "championId": 76, "participantId": 5, "player": null, "role": "NONE", "summonerId": 289863}, {"lane": "BOTTOM", "championId": 21, "participantId": 6, "player": null, "role": "DUO_CARRY", "summonerId": 2952658}, {"lane": "MIDDLE", "championId": 25, "participantId": 7, "player": null, "role": "SOLO", "summonerId": 271698}, {"lane": "BOTTOM", "championId": 78

In [57]:
print match.id

155530071


In [4]:
test_summoner = masters[0].summoner
print test_summoner.name
print test_summoner.id
print test_summoner.data.summonerLevel
print test_summoner.data.revisionDate
print league_dict.tier
print test_summoner.data.division

NameError: name 'masters' is not defined

In [35]:
summoner_dict = dict()
summoner_dict['name'] = test_summoner.name
summoner_dict['id'] = test_summoner.id
summoner_dict['summonerLevel'] = test_summoner.data.summonerLevel
summoner_dict['revisionDate'] = test_summoner.data.revisionDate

In [76]:
print summoners_collection.find_one({})

None


In [47]:
summoners_collection.insert_one(summoner_dict)

In [48]:
#show all entries
for entry in summoners_collection.find():
    print entry

{u'id': 372999, u'_id': ObjectId('583502eb666d59386fcd9192'), u'revisionDate': 1479534704000L, u'name': u'cat and bird', u'summonerLevel': 30}


In [49]:
#delete all entries
summoners_collection.delete_many({})